In [10]:
import pandas as pd
import os
import datetime
import re
from datetime import datetime
import pickle as pkl

In [2]:
df = pd.read_csv('travelblog_data_100samples.csv', error_bad_lines=False)

b'Skipping line 84: expected 7 fields, saw 8\nSkipping line 197: expected 7 fields, saw 9\nSkipping line 330: expected 7 fields, saw 18\nSkipping line 572: expected 7 fields, saw 9\nSkipping line 717: expected 7 fields, saw 8\nSkipping line 807: expected 7 fields, saw 9\nSkipping line 815: expected 7 fields, saw 8\nSkipping line 856: expected 7 fields, saw 8\nSkipping line 926: expected 7 fields, saw 9\nSkipping line 1378: expected 7 fields, saw 9\nSkipping line 1382: expected 7 fields, saw 8\nSkipping line 1496: expected 7 fields, saw 10\nSkipping line 1728: expected 7 fields, saw 10\nSkipping line 1754: expected 7 fields, saw 8\nSkipping line 1803: expected 7 fields, saw 8\nSkipping line 1853: expected 7 fields, saw 11\nSkipping line 2022: expected 7 fields, saw 8\nSkipping line 2029: expected 7 fields, saw 9\nSkipping line 2072: expected 7 fields, saw 8\nSkipping line 2076: expected 7 fields, saw 8\nSkipping line 2204: expected 7 fields, saw 11\nSkipping line 2277: expected 7 fields

In [3]:
df['Combined'] = df[df.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
df.drop(columns = ['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href','title', 'date', 'text;;;;;;;;;'], inplace = True)

In [4]:
# combining multiple rows to one to get one blogpost starting always with a 14 length web scraper order  
indices = []
for row in range(len(df)):
    web_id  = df["Combined"][row].split(",")[0]
    if len(web_id) == 14 and bool(re.match('[\d/-]+$', web_id)) == True :
        indices.append(row)

df_new = pd.DataFrame(columns=['Combined'])

for i,k in zip(indices[0::2], indices[1::2]):
    
    new_row = df.iloc[i]
    for j in range(i+1, k):
    
        new_row = new_row + ' ' + df.iloc[j]
    df_new = df_new.append(new_row, ignore_index=True)



In [5]:
# creating correctly structured df 
df_final = pd.DataFrame(columns=['web-scraper-order', 'web-scraper-start-url', 'link', 'link-href','title', 'date', 'text'])

for i in range(len(df_new)):
    df_final = df_final.append({'web-scraper-order': list(df_new.loc[i].values)[0].split(',"')[0], 
                               'web-scraper-start-url': list(df_new.loc[i].values)[0].split(',"')[1],
                               'link': list(df_new.loc[i].values)[0].split(',"')[2],
                               'link-href': list(df_new.loc[i].values)[0].split(',"')[3],
                               'title': list(df_new.loc[i].values)[0].split(',"')[4],
                               'date': list(df_new.loc[i].values)[0].split(',"')[5],
                               'text': list(df_new.loc[i].values)[0].split(',"')[6]}, ignore_index=True)

In [6]:
# cleaning 
df_final.date = df_final.date.str.replace('"', '')
df_final.title = df_final.title.str.replace('"', '')
df_final['web-scraper-start-url'] = df_final['web-scraper-start-url'].str.replace('"', '')
df_final.link = df_final.link.str.replace('"', '')
df_final['link-href'] = df_final['link-href'].str.replace('"', '')
df_final.text = df_final.text.str.replace(';', '')
df_final.text = df_final.text.str.replace('\xa0', '')

In [7]:
# replace string date to datetime 

lst = [datetime.strptime(df_final.date[i], '%B %d, %Y') for i in range(len(df_final))]
df_final.date = lst

In [8]:
df_final

,web-scraper-order,web-scraper-start-url,link,link-href,title,date,text
0,1623408047-415,https://www.neverendingfootsteps.com/blog/page/17,Conquering My Fear of Sailing in Greece,https://www.neverendingfootsteps.com/conquerin...,Conquering My Fear of Sailing in Greece,2020-10-04,I didn’t developa fear of sailing until I actu...
1,1623407936-381,https://www.neverendingfootsteps.com/blog/page/20,A Month Spent Eating Everything in Brixton,https://www.neverendingfootsteps.com/where-to-...,A Month Spent Eating Everything in Brixton,2021-01-14,"When I set off to travel the world, I confesse..."
2,1623408025-408,https://www.neverendingfootsteps.com/blog/page/18,The Cost of Travel in Cambodia: A Detailed Bud...,https://www.neverendingfootsteps.com/cost-of-t...,The Cost of Travel in Cambodia: A Detailed Bud...,2021-01-14,Cambodia is such a wonderful country. I...
3,1623408058-419,https://www.neverendingfootsteps.com/blog/page/17,That Time I Stayed in a Hotel Without Walls,https://www.neverendingfootsteps.com/hotel-wit...,That Time I Stayed in a Hotel Without Walls,2020-08-15,"In case you hadn’t noticed, one of my favourit..."
4,1623407973-392,https://www.neverendingfootsteps.com/blog/page/19,Month 53: Travel Summary and Statistics,https://www.neverendingfootsteps.com/month-53-...,Month 53: Travel Summary and Statistics,2021-01-14,"Greetings from scorching hot Melbourne, where ..."
5,1623408158-449,https://www.neverendingfootsteps.com/blog/page/14,"My Travel, Work, and Health Goals for 2017",https://www.neverendingfootsteps.com/travel-wo...,"My Travel, Work, and Health Goals for 2017",2018-05-29,It’s that time of year again! …She says as if ...
6,1623408005-402,https://www.neverendingfootsteps.com/blog/page/19,What’s it Like to Travel in French Polynesia?,https://www.neverendingfootsteps.com/whats-it-...,What’s it Like to Travel in French Polynesia?,2021-03-24,"French Polynesia is paradise. I mean, just loo..."
7,1623408153-447,https://www.neverendingfootsteps.com/blog/page/14,2016: My Year in Review,https://www.neverendingfootsteps.com/2016-trav...,2016: My Year in Review,2020-10-04,"During the first few momentsof 2016, I turned ..."
8,1623408033-411,https://www.neverendingfootsteps.com/blog/page/18,The Worst Travel Day Ever,https://www.neverendingfootsteps.com/the-worst...,The Worst Travel Day Ever,2020-08-15,"When I first started travelling, my priority w..."
9,1623408044-414,https://www.neverendingfootsteps.com/blog/page/17,Month 55: Travel Summary and Statistics,https://www.neverendingfootsteps.com/month-55-...,Month 55: Travel Summary and Statistics,2019-10-07,My monthly summaries are back by popular deman...


In [12]:
pkl.dump(df_final("travel_blog_data.pkl"),"wb")

TypeError: 'DataFrame' object is not callable